In [120]:
# import here
import pandas as pd
from sqlalchemy import create_engine
from references_dict import Team_Dictionary

# connect to database
engine = create_engine('mysql+pymysql://root:@localhost:3306/nfl_db')
conn = engine.connect()

In [121]:
seasons_sql = "select distinct(schedule_season) from nfl_gameinfo"
seasons = pd.read_sql_query(seasons_sql,conn,index_col=None)
seasons = seasons['schedule_season'].tolist()
seasons

[2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]

In [122]:
teams_sql = "select distinct(home_abbrev) from nfl_gameinfo where schedule_season = %s"
values = (2009,)
teams = pd.read_sql_query(teams_sql,conn,params=values)
teams = teams['home_abbrev'].tolist()
teams

['PIT',
 'ARI',
 'ATL',
 'BAL',
 'CAR',
 'CIN',
 'CLE',
 'GB',
 'HOU',
 'IND',
 'NO',
 'NYG',
 'SEA',
 'TB',
 'NE',
 'OAK',
 'BUF',
 'CHI',
 'DAL',
 'DEN',
 'DET',
 'JAX',
 'KC',
 'NYJ',
 'PHI',
 'SD',
 'SF',
 'TEN',
 'WAS',
 'MIA',
 'MIN',
 'STL']

In [123]:
df = pd.DataFrame(columns=['game_id', 'team', 'rush_skillpoints','pass_skillpoints','season_avg_rush_points','season_avg_pass_points'])
for season in seasons:
    teams_sql = "select distinct(home_abbrev) from nfl_gameinfo where schedule_season = %s"
    values = (season,)
    teams = pd.read_sql_query(teams_sql,conn,params=values)
    teams = teams['home_abbrev'].tolist()
    for team in teams:
        team_sql = "select game_id, \
          %s as team, \
          case \
            when home_abbrev=%s then rush_skillpoints_home \
            else rush_skillpoints_away \
            end as rush_skillpoints, \
          case \
            when home_abbrev=%s then pass_skillpoints_home \
            else pass_skillpoints_away \
          end as pass_skillpoints \
          from nfl_gameinfo \
          where schedule_season=%s \
          and( home_abbrev=%s or away_abbrev=%s) \
          order by cast(schedule_week as int)"
        values = (team,team,team,season,team,team)
        print("Getting stats for ",season," - ",team)
        team_stats = pd.read_sql_query(team_sql,conn,params=values,index_col=None)
        i = 1
        rush_avg = []
        pass_avg = []
        for idx,row in team_stats.iterrows():
            rush_avg.append(team_stats['rush_skillpoints'].iloc[:i].mean())
            pass_avg.append(team_stats['pass_skillpoints'].iloc[:i].mean())
            i += 1
        team_stats['season_avg_rush_points'] = rush_avg
        team_stats['season_avg_pass_points'] = pass_avg
        print(team_stats)
        df = df.append(team_stats, sort=True,ignore_index=True)

Getting stats for  2009  -  PIT
         game_id team  rush_skillpoints  pass_skillpoints  \
0   2.009091e+09  PIT          5.165217         57.470213   
1   2.009092e+09  PIT         21.647619         40.794444   
2   2.009093e+09  PIT         19.842857         45.725000   
3   2.009100e+09  PIT         35.164706         68.602703   
4   2.009101e+09  PIT         19.222222         57.231250   
5   2.009102e+09  PIT         24.633333         62.621053   
6   2.009103e+09  PIT         15.900000         27.500000   
7   2.009111e+09  PIT         24.118519         52.525806   
8   2.009112e+09  PIT         12.444444         20.490909   
9   2.009112e+09  PIT         15.077419         75.670213   
10  2.009113e+09  PIT         25.326316         24.038462   
11  2.009121e+09  PIT         24.415385         49.760000   
12  2.009121e+09  PIT         10.909091         20.200000   
13  2.009122e+09  PIT         15.921053         79.790196   
14  2.009123e+09  PIT         13.650000         35.92

         game_id team  rush_skillpoints  pass_skillpoints  \
0   2.009091e+09   NE         16.473913         73.960714   
1   2.009092e+09   NE         12.450000         31.100000   
2   2.009093e+09   NE         27.594595         48.580952   
3   2.009100e+09   NE         23.807143         41.400000   
4   2.009101e+09   NE         13.430769         43.117647   
5   2.009102e+09   NE         39.407407        107.923404   
6   2.009103e+09   NE         21.230769         59.830303   
7   2.009111e+09   NE         21.441667         50.825000   
8   2.009112e+09   NE         21.622222         70.613636   
9   2.009112e+09   NE         26.271429         53.232558   
10  2.009113e+09   NE         28.557143         32.195238   
11  2.009121e+09   NE         19.440000         55.648276   
12  2.009121e+09   NE         29.621053         34.481250   
13  2.009122e+09   NE         20.812903         20.760870   
14  2.009123e+09   NE         31.172222         71.046154   
15  2.010010e+09   NE   

         game_id team  rush_skillpoints  pass_skillpoints  \
0   2.009091e+09  WAS         18.547619         34.548276   
1   2.009092e+09  WAS         17.200000         36.997297   
2   2.009093e+09  WAS         11.142857         59.357143   
3   2.009100e+09  WAS         17.193750         28.000000   
4   2.009101e+09  WAS         16.483333         24.021429   
5   2.009102e+09  WAS         17.700000         19.300000   
6   2.009103e+09  WAS          9.463158         49.563265   
7   2.009111e+09  WAS         23.850000         31.081250   
8   2.009112e+09  WAS         27.750000         42.972414   
9   2.009112e+09  WAS         11.700000         37.757895   
10  2.009113e+09  WAS         17.480000         42.836842   
11  2.009121e+09  WAS         12.608333         74.128571   
12  2.009121e+09  WAS         26.140000         39.458065   
13  2.009122e+09  WAS         19.844444         27.710811   
14  2.009123e+09  WAS          6.688889         28.948780   
15  2.010010e+09  WAS   

         game_id team  rush_skillpoints  pass_skillpoints  \
0   2.010091e+09  SEA         17.047826         39.500000   
1   2.010092e+09  SEA         22.789474         35.728571   
2   2.010093e+09  SEA         10.350000         29.517647   
3   2.010100e+09  SEA          9.066667         26.656098   
4   2.010102e+09  SEA         27.471429         45.825000   
5   2.010102e+09  SEA         18.642857         29.502326   
6   2.010103e+09  SEA          7.173684         10.225000   
7   2.010111e+09  SEA          8.400000         19.560870   
8   2.010111e+09  SEA         27.622222         56.695238   
9   2.010112e+09  SEA          9.211765         65.313953   
10  2.010113e+09  SEA          4.400000         39.526316   
11  2.010121e+09  SEA         39.600000         24.657576   
12  2.010122e+09  SEA         25.800000         18.400000   
13  2.010123e+09  SEA         24.470370         13.538462   
14  2.011010e+09  SEA         18.633333         38.644444   

    season_avg_rush_poi

Getting stats for  2010  -  DET
         game_id team  rush_skillpoints  pass_skillpoints  \
0   2.010091e+09  DET         14.509524         24.528571   
1   2.010092e+09  DET         28.240000         52.520833   
2   2.010093e+09  DET          9.615789         45.400000   
3   2.010100e+09  DET         18.157143         58.380702   
4   2.010101e+09  DET         13.200000         64.514286   
5   2.010102e+09  DET          9.447619         55.300000   
6   2.010103e+09  DET         14.280000         56.283333   
7   2.010111e+09  DET         16.585714         44.256410   
8   2.010111e+09  DET         10.523077         51.473077   
9   2.010112e+09  DET         11.250000         57.233333   
10  2.010112e+09  DET         29.677778         43.687500   
11  2.010121e+09  DET         24.362963         31.646154   
12  2.010121e+09  DET         23.634146         15.866667   
13  2.010122e+09  DET         30.564286         45.497297   
14  2.010123e+09  DET         15.890476         39.20

...

         game_id team  rush_skillpoints  pass_skillpoints  \
0   2.017091e+09   SF          8.500000         28.069231   
1   2.017092e+09   SF         24.268421         13.935714   
2   2.017092e+09   SF         32.724242         51.795238   
3   2.017100e+09   SF         13.018519         29.476923   
4   2.017101e+09   SF          9.600000         63.193617   
5   2.017102e+09   SF         24.750000         39.480000   
6   2.017102e+09   SF         20.981818         29.755814   
7   2.017103e+09   SF         13.316667         21.925000   
8   2.017111e+09   SF         16.005882         32.685714   
9   2.017111e+09   SF         37.200000         53.240000   
10  2.017113e+09   SF         11.900000         36.948780   
11  2.017121e+09   SF         20.385714         49.521212   
12  2.017122e+09   SF          7.418182         62.455556   
13  2.017122e+09   SF         35.082353         47.425806   
14  2.017123e+09   SF         35.342857         49.321212   

    season_avg_rush_poi

         game_id team  rush_skillpoints  pass_skillpoints  \
0   2.017091e+09  OAK         14.928571         50.435294   
1   2.017092e+09  OAK         42.666667         59.892857   
2   2.017092e+09  OAK          6.050000         20.114286   
3   2.017100e+09  OAK          4.000000         37.505263   
4   2.017101e+09  OAK         21.120000         24.727586   
5   2.017102e+09  OAK         21.854545         34.800000   
6   2.017102e+09  OAK         18.990476         75.873077   
7   2.017103e+09  OAK         15.257143         55.320833   
8   2.017111e+09  OAK         24.325000         47.225806   
9   2.017112e+09  OAK         16.090476         42.380000   
10  2.017113e+09  OAK         19.847059         47.761538   
11  2.017120e+09  OAK         28.285714         46.781081   
12  2.017121e+09  OAK         19.363636         32.900000   
13  2.017122e+09  OAK         17.504348         40.705263   
14  2.017123e+09  OAK         18.112500         22.500000   
15  2.017123e+09  OAK   

         game_id team  rush_skillpoints  pass_skillpoints  \
0   2.018091e+09  CAR         32.400000         23.421429   
1   2.018092e+09  CAR         18.822222         69.287234   
2   2.018092e+09  CAR         40.875000         35.375000   
3   2.018101e+09  CAR         15.866667         43.300000   
4   2.018101e+09  CAR         13.023529         54.280488   
5   2.018102e+09  CAR         25.825000         52.143902   
6   2.018103e+09  CAR         32.632258         50.812903   
7   2.018110e+09  CAR         48.700000         48.584615   
8   2.018111e+09  CAR         35.318182         79.717647   
9   2.018112e+09  CAR          9.100000         63.580952   
10  2.018113e+09  CAR         36.148148         56.433333   
11  2.018120e+09  CAR         26.288889         47.043478   
12  2.018121e+09  CAR         25.440000         43.368182   
13  2.018122e+09  CAR         14.060870         26.857576   

    season_avg_rush_points  season_avg_pass_points  
0                32.400000     

         game_id team  rush_skillpoints  pass_skillpoints  \
0   2.018091e+09  CIN         21.150000         47.000000   
1   2.018091e+09  CIN         14.657143         64.214286   
2   2.018092e+09  CIN         17.676923         55.093617   
3   2.018093e+09  CIN         26.204348         65.813636   
4   2.018101e+09  CIN         14.560000         39.300000   
5   2.018101e+09  CIN         16.969231         46.922222   
6   2.018102e+09  CIN          9.921053         31.014286   
7   2.018103e+09  CIN         31.246154         50.600000   
8   2.018111e+09  CIN         40.875000         50.676923   
9   2.018112e+09  CIN         13.800000         41.600000   
10  2.018113e+09  CIN         25.350000         52.700000   
11  2.018120e+09  CIN         15.926087         39.225000   
12  2.018121e+09  CIN         24.900000         30.125000   
13  2.018122e+09  CIN         33.978947         21.764706   

    season_avg_rush_points  season_avg_pass_points  
0                21.150000     

Getting stats for  2018  -  HOU
         game_id team  rush_skillpoints  pass_skillpoints  \
0   2.018091e+09  HOU         27.890909         26.410811   
1   2.018092e+09  HOU         20.492308         50.828571   
2   2.018092e+09  HOU          9.005263         59.214286   
3   2.018093e+09  HOU         21.300000         58.393617   
4   2.018101e+09  HOU         11.638710         64.700000   
5   2.018101e+09  HOU         10.760870         19.518182   
6   2.018102e+09  HOU         24.385714         24.660000   
7   2.018102e+09  HOU         30.458824         66.700000   
8   2.018110e+09  HOU         12.769697         39.621429   
9   2.018112e+09  HOU         18.666667         29.281481   
10  2.018113e+09  HOU         48.745455         42.370370   
11  2.018120e+09  HOU         23.747368         38.228571   
12  2.018121e+09  HOU         24.460000         44.653488   
13  2.018122e+09  HOU          7.800000         49.635294   

    season_avg_rush_points  season_avg_pass_points  

In [124]:
df.head()

,game_id,pass_skillpoints,rush_skillpoints,season_avg_pass_points,season_avg_rush_points,team
0,2.009091e+09,57.470213,5.165217,57.470213,5.165217,PIT
1,2.009092e+09,40.794444,21.647619,49.132329,13.406418,PIT
2,2.009093e+09,45.725000,19.842857,47.996552,15.551898,PIT
3,2.009100e+09,68.602703,35.164706,53.148090,20.455100,PIT
4,2.009101e+09,57.231250,19.222222,53.964722,20.208524,PIT


In [125]:
df.to_sql('nfl_skillpoints', con=engine, if_exists='replace',index=False)

In [2]:
gameinfo = open('game_mets.sql', 'r')
df = pd.read_sql_query(gameinfo.read(),conn, index_col='game_id')
df.head()

,spread_favorite,over_under_line,home_favorite,spread_result,OU_result,fav_rush_yds,fav_rush_att,fav_rush_tds,fav_short_pass_yds,fav_deep_pass_yds,...,und_rush_yds,und_rush_att,und_rush_tds,und_short_pass_yds,und_deep_pass_yds,und_pass_tds,und_pass_att,und_completions,und_sacked,und_interceptions
game_id,,,,,,,,,,,,,,,,,,,,,
2.009091e+09,-6.5,35,1,-1,-1,36.0,23.0,0.0,218.0,145.0,...,86.0,25.0,0.0,152.0,92.0,1.0,36.0,22.0,1.0,1.0
2.009091e+09,-4.5,45,1,-1,-1,40.0,17.0,0.0,223.0,65.0,...,21.0,25.0,1.0,159.0,50.0,1.0,35.0,18.0,4.0,0.0
2.009091e+09,-4.0,44.5,1,1,-1,72.0,24.0,0.0,207.0,22.0,...,96.0,22.0,0.0,139.0,37.0,1.0,33.0,21.0,4.0,1.0
2.009091e+09,-13.0,36.5,1,1,1,198.0,41.0,2.0,195.0,112.0,...,29.0,17.0,0.0,127.0,50.0,2.0,27.0,16.0,3.0,0.0
2.009091e+09,-2.5,43.5,0,1,1,186.0,31.0,1.0,102.0,0.0,...,87.0,29.0,1.0,94.0,39.0,0.0,39.0,14.0,5.0,5.0
